In [1]:
import pyaudio
import struct
import numpy as np
import tensorflow as tf
import librosa
import wave
from scipy.io import wavfile
import IPython.display as ipd
import noisereduce as nr
print("completed")

completed


In [2]:
# Load the trained model
model = tf.keras.models.load_model('C:\\Users\\gana4\\MAJOR PROJECT\\Speech recognition major module2.h5')

In [3]:
def recordAudio():
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    fs = 48100  # Record at 44100 samples per second //as per ravdess dataset the frequecy is 48kHz
    seconds = 5
    filename = "Predict-Record-Audio.wav"

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 10 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    #noise reduction
    # load data
    rate, data = wavfile.read("Predict-Record-Audio.wav")
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write("mywav_reduced_noise.wav", rate, reduced_noise)

In [4]:
def extract_mfcc(wav_file_name):
    y, sr = librosa.load(wav_file_name)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
    return mfccs

In [5]:
wav_filepath = "C:\\Users\\gana4\\MAJOR PROJECT\\mywav_reduced_noise.wav"

In [6]:
def Speechrecognition():
    emotions={1 : 'neutral', 2 : 'calm', 3 : 'happy', 4 : 'sad', 5 : 'angry', 6 : 'fearful', 7 : 'disgust', 8 : 'surprised'}
    test_point=extract_mfcc(wav_filepath)
    test_point=np.reshape(test_point,newshape=(1,40,1))
    predictions=model.predict(test_point)
    varemotion = emotions[np.argmax(predictions[0] )+1]
    return varemotion

In [7]:
recordAudio()
Speechrecognition()

Recording
Finished recording
1/1 [==============================] - 4s 4s/step


'happy'

In [8]:
#original audio
wav_filepath = "C:\\Users\\gana4\\MAJOR PROJECT\\Predict-Record-Audio.wav"
Speechrecognition()
ipd.Audio('Predict-Record-Audio.wav')

1/1 [==============================] - 0s 31ms/step


In [9]:
#after noise reduction
wav_filepath = "C:\\Users\\gana4\\MAJOR PROJECT\\mywav_reduced_noise.wav"
Speechrecognition()
ipd.Audio('mywav_reduced_noise.wav')

1/1 [==============================] - 0s 32ms/step
